In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

In [2]:
df = pd.read_csv('data/matrix_intDef_ec_propensity.tab', sep='\t')
df.head()

,ensID,pdbID,pos.ens,pos.pdb,aa,ASA.rel.cplx,ASA.rel.alone,nsub,sym,len,homo,cat,patch.alone.size,patch.cplx.size,patch.alone,patch.cplx
0,P76342,1xdy_5E,63,19,A,112.4,112.4,1,NPS,262,1.0,2,7,7,0.4348,0.063
1,P76342,1xdy_5E,64,20,L,19.3,19.3,1,NPS,262,1.0,1,0,0,-0.9138,0.000
2,P76342,1xdy_5E,65,21,E,98.6,98.6,1,NPS,262,1.0,2,10,10,-1.0007,-0.179
3,P76342,1xdy_5E,66,22,F,29.3,29.3,1,NPS,262,1.0,2,11,11,-2.2957,-0.093
4,P76342,1xdy_5E,67,23,S,54.6,54.6,1,NPS,262,1.0,2,8,8,-1.4256,-0.161


In [3]:
#taking only the surface aas
df2 = df.loc[df['cat']==2]
df2.head()

,ensID,pdbID,pos.ens,pos.pdb,aa,ASA.rel.cplx,ASA.rel.alone,nsub,sym,len,homo,cat,patch.alone.size,patch.cplx.size,patch.alone,patch.cplx
0,P76342,1xdy_5E,63,19,A,112.4,112.4,1,NPS,262,1.0,2,7,7,0.4348,0.063
2,P76342,1xdy_5E,65,21,E,98.6,98.6,1,NPS,262,1.0,2,10,10,-1.0007,-0.179
3,P76342,1xdy_5E,66,22,F,29.3,29.3,1,NPS,262,1.0,2,11,11,-2.2957,-0.093
4,P76342,1xdy_5E,67,23,S,54.6,54.6,1,NPS,262,1.0,2,8,8,-1.4256,-0.161
5,P76342,1xdy_5E,68,24,K,58.7,58.7,1,NPS,262,1.0,2,10,10,-0.3994,-0.158


In [4]:
#stickiness score of aa
st = pd.read_csv('data/CROWDING_EC.mat', sep='\t')
st.sort_index(inplace=True)
st = st.reset_index(drop=True)
st.head()

,ensID,pdbID,pos.ens,pos.pdb,aa,rate,ndef,ASA.rel.cplx,ASA.rel.alone,len,patch.compo.400abs,aa.prop,ab.all
0,P76342,1xdy_5E,141,97,I,0.6596,10,7.1,7.1,262,0.0,1.1109,NaN
1,P76342,1xdy_5E,142,98,Y,0.4077,10,0.1,0.1,262,0.0,0.8806,NaN
2,P76342,1xdy_5E,143,99,R,0.2909,10,21.3,21.3,262,0.0,-0.0876,NaN
3,P76342,1xdy_5E,144,100,M,0.8333,10,5.3,5.3,262,0.0,1.0124,NaN
4,P76342,1xdy_5E,140,96,R,0.2909,10,4.2,4.2,262,0.0,-0.0876,NaN


In [5]:
merged = pd.merge(df, st, on=['ensID', 'pos.ens'])


In [6]:
#merged_with_surface = df2.merge(st, on=['ensID', 'aa'])
merged_with_surface = merged.loc[merged['cat'] == 2].copy()

In [7]:
aa_prop = merged_with_surface['aa_x'].groupby([st.ensID, st.pdbID]).apply(list).reset_index()
aa_prop.head()

,ensID,pdbID,aa_x
0,O32583,1zud4,"[G, Q, D, E, K, S, L, L, E, D, T, T, E, G, K, ..."
1,P00448,1ixbB,"[D, E, P, A, K, D, K, G, K, A, A, Y, A, K, N, ..."
2,P00452,1r1r_2B,"[L, E, H, D, K, D, K, D, V, V, G, M, N, R, L, ..."
3,P00509,1x2aB,"[N, A, P, A, L, R, A, E, R, P, G, K, D, E, T, ..."
4,P00803,1b12_3C,"[E, K, I, K, K, M, D, L, L, H, K, G, T, H, E, ..."


In [8]:
aa_prop['Surface'] = aa_prop['aa_x'].apply(lambda x:''.join(x))

# Merge with ecoli

In [9]:
ecoli_w3110 = pd.read_csv('data/ecoli_eSOL_JWXXXX.csv')
ecoli_w3110[['JW', 'name', 'ECK_num']] = ecoli_w3110['Accession'].str.split(' ', expand=True)

#uniprot
a = pd.read_csv('data/ecoli_mapping.txt',sep='\t', header=None)
b = pd.DataFrame()
b[[1,2,3,4,5]] = a[0].str.split(' ', 4, expand=True)
b[['b_num', 'JW']] = b[1].str.split(';', 1, expand=True)
b = b.rename(columns={2:'Swiss-prot_entry_name', 3:'ensID', 4:'Length', 5:'Gene_name_and_synonyms'})
mapping = b[['Swiss-prot_entry_name', 'ensID','Length', 'Gene_name_and_synonyms', 'b_num', 'JW']].copy()


In [10]:
mapping.to_pickle('mapping.pkl')

In [11]:
mapped_df = pd.merge(aa_prop, mapping, on=['ensID']) #first map aa_prop
final_df = pd.merge(mapped_df, ecoli_w3110, on=['JW']) #then map with ecoli
final_df.head(1)

,ensID,pdbID,aa_x,Surface,Swiss-prot_entry_name,Length,Gene_name_and_synonyms,b_num,JW,Accession,Sequence,ECK number,Solubility(%),unknown_bases,name,ECK_num
0,P00452,1r1r_2B,"[L, E, H, D, K, D, K, D, V, V, G, M, N, R, L, ...",LEHDKDKDVVGMNRLWSGKESLDDLDNNKEQDETGGNQDRHRQGDN...,RIR1_ECOLI,761,nrdA;dnaF,b2234,JW2228,JW2228 nrdA ECK2226,MNQNLLVTKRDGSTERINLDKIHRVLDWAAEGLHNVSISQVELRSH...,ECK2226,63.0,False,nrdA,ECK2226


In [12]:
final_df.to_pickle('surface_aminoacids.pkl.gz', compression='infer')

In [13]:
df_opt = final_df[['Surface', 'Sequence', 'Solubility(%)']].copy()

# Optimisation based on surface aminoacids

In [14]:
from scipy import optimize
from functools import partial
np.random.seed(12345)


flexibilities_smith = {'A': 0.717, 'C': 0.668, 'E': 0.963, 'D': 0.921,\
                         'G': 0.843, 'F': 0.599, 'I': 0.632, 'H': 0.754, \
                         'K': 0.912, 'M': 0.685, 'L': 0.681, 'N': 0.851,\
                         'Q': 0.849, 'P': 0.85, 'S': 0.84, 'R': 0.814, 'T': 0.758, \
                         'W': 0.626, 'V': 0.619, 'Y': 0.615}


#SWI
swi = {'A': 0.8356956599678218,
 'C': 0.5219207324456876,
 'E': 0.9868660417547442,
 'D': 0.9075983546378998,
 'G': 0.8003827946673535,
 'F': 0.5821934635876957,
 'I': 0.6790449304566072,
 'H': 0.8963977585570367,
 'K': 0.9259165090012061,
 'M': 0.6299964100098959,
 'L': 0.6546922237065839,
 'N': 0.8604957042204235,
 'Q': 0.7895650031998229,
 'P': 0.822104415564934,
 'S': 0.7442464390120463,
 'R': 0.771055152304471,
 'T': 0.8098670971949234,
 'W': 0.6386931894494416,
 'V': 0.7344952876686051,
 'Y': 0.6125581495225544}

def avaas(seq, flex):
    w = []
    for i, v in enumerate(seq):
        w.append(flex[v])
#     return np.exp(np.sum(np.log(w))/len(w)) #similar to CAI
    return w


def make_dic(arr):
    '''Make an amino acid dictionary from an array of values
    '''
    dic = {}
    ks = [k for k, v in flexibilities_smith.items()]
    for i, v in enumerate(ks):
        dic[v] = arr[i]
    return dic



def cost_func(f, df):
    #based on surface amino acids only
    #use for optimisation because p value isnt needed
    flex = make_dic(f)
    df['f'] = df['Surface'].apply(lambda x:avaas(x, flex))
    df['Average_flexibility'] = df['f'].apply(lambda x:np.mean(x))
    corr, pval = spearmanr(df['Average_flexibility'],df['Solubility(%)'])
    return -corr


def cost_func_p_val(f, df):
    #based on surface amino acids only, but returns p value as well
    flex = make_dic(f)
    df['f'] = df['Surface'].apply(lambda x:avaas(x, flex))
    df['Average_flexibility'] = df['f'].apply(lambda x:np.mean(x))
    corr, pval = spearmanr(df['Average_flexibility'],df['Solubility(%)'])
    return corr, pval


def cost_func_full(f, df):
    #based on full sequence
    flex = make_dic(f)
    df['f'] = df['Sequence'].apply(lambda x:avaas(x, flex))
    df['Average_flexibility'] = df['f'].apply(lambda x:np.mean(x))
    corr, pval = spearmanr(df['Average_flexibility'],df['Solubility(%)'])
    return corr, pval


In [15]:
#Optimise weights based on b-factors


init_state = [v for k, v in flexibilities_smith.items()]
cost_function = partial(cost_func, df=df_opt)

#Optimise cost function using init_state as starting point
optimize.minimize(cost_function, init_state, method="COBYLA", \
                  options={'rhobeg': 0.001, 'maxiter': 500, 'disp': True, 'catol': 0.0002})

     fun: -0.057968389938518695
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 118
  status: 1
 success: True
       x: array([0.71828685, 0.66909515, 0.96419968, 0.92115128, 0.84519299,
       0.59899378, 0.63189739, 0.75616006, 0.91355243, 0.68464218,
       0.68026704, 0.85183207, 0.84803086, 0.85105998, 0.84046509,
       0.81369212, 0.75841106, 0.62586971, 0.61762375, 0.61563007])

In [16]:
#find correlation based on weights optimised based on surface amino acids
optimised_weights = [0.71828685, 0.66909515, 0.96419968, 0.92115128, 0.84519299,
       0.59899378, 0.63189739, 0.75616006, 0.91355243, 0.68464218,
       0.68026704, 0.85183207, 0.84803086, 0.85105998, 0.84046509,
       0.81369212, 0.75841106, 0.62586971, 0.61762375, 0.61563007]

cost_func_p_val(optimised_weights, df_opt)

(0.057968389938518695, 0.2808541175490529)

In [17]:
#using SWI for full length not just surface amino acids

init_state =  [v for k, v in swi.items()]

cost_func_full(init_state, df_opt)

(0.45444827397772025, 3.884659642543914e-19)

## Conclusion
Optimisation based on surface amino acids didnt lead to any significant improvement in correlation with solubility. However, if we take full sequence into consideration and use AAS, correlation is much improved (0.45, p-val 3.88 x 10^(-19))